In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants
import scipy.optimize
import scipy.signal

In [ ]:
#wvl = 1500e-6
#freq = constants.c*2*np.pi/(wvl)
#print("{:.3f} GHz".format(freq/(2*np.pi*1e9)))



def carrier_signal(frequency, time):
    return np.sin(2*np.pi*frequency*time) 

def modulation_signal(frequency, time, modulation):
    phase = 0.
    #modulation = modulation*np.concatenate([np.linspace(0., 1., int((time.size)/2)),
    #                                        np.linspace(1., 0., int((time.size)/2))])
    signal = modulation*np.sin(2*np.pi*(frequency)*time+phase)
    return signal

def combine_signals(signal1, signal2):
    combined_signal = (1+signal2)*signal1
    #combined_signal = signal1 + signal2
    return combined_signal

def combine_signals_digital(signal1, signal2):
    combined_signal = (signal2)*signal1
    #combined_signal = signal1 + signal2
    return combined_signal


def demodulate(combined_signal, carrier):
    combined_signal = np.abs(combined_signal)
    sos = scipy.signal.butter(10, 50, 'lp', fs=1000, output='sos')
    filtered = scipy.signal.sosfilt(sos, combined_signal)
    return filtered

def add_noise(signal, noise_level):
    noise = np.random.normal(0, noise_level, signal.size).reshape(signal.shape)
    return signal+noise

def model_func(model_inputs):
    carrier = carrier_signal(model_inputs[0], model_inputs[1])
    signal = modulation_signal(model_inputs[0], model_inputs[1], model_inputs[2])
    combined_signal = combine_signals(carrier, signal)
    return combined_signal

def error_func(x, model_inputs, exp_data):
    model_inputs[2] = x
    model_data = model_func(model_inputs)
    return exp_data-model_data

In [ ]:
fig, [ax0, ax1, ax2, ax3] = plt.subplots(4,1, figsize=(12,12))
signal_freq = 10.
carrier_freq = 100.

t = np.linspace(0, 1., 1000, False)
carrier = carrier_signal(carrier_freq, t)
plt.sca(ax0)
plt.plot(t, carrier)

modulation = 1.0

signal = modulation_signal(signal_freq, t, modulation)
plt.sca(ax1)
plt.plot(t, signal)

combined_signal = combine_signals(carrier, signal)
noisy_signal = add_noise(combined_signal, 1.0)
plt.sca(ax2)
plt.plot(t, noisy_signal)


#exp_data = noisy_signal
#model_inputs = [ang_freq, t, 0.0]
#res = scipy.optimize.least_squares(error_func, [0.0], args=(model_inputs, exp_data))
#print(res['x'])
#reconstructed_signal =  modulation_signal(ang_freq, t, res['x'])
#plt.plot(t, 1+reconstructed_signal, ls='--')
#plt.plot(t, -(1+reconstructed_signal), ls='--')

plt.sca(ax3)
demodulated_signal = demodulate(noisy_signal, carrier)
plt.plot(t, demodulated_signal)

In [ ]:
fig, [ax0, ax1, ax2, ax3] = plt.subplots(4,1, figsize=(12,12))
signal_freq = 10.
carrier_freq = 100.

t = np.linspace(0, 1., 1000, False)
carrier = carrier_signal(carrier_freq, t)
plt.sca(ax0)
plt.plot(t, carrier)

modulation = 1.0

def smooth_step (edge0, edge1, t):
    f = np.zeros_like(t)
    f[t>edge1] = 1.0    
    #// Scale/bias into [0..1] range    
    mask = t>=edge0
    mask *= t<=edge1
    scaled_t = (t[mask]-edge0)/(edge1-edge0)    
    f[mask] = scaled_t * scaled_t * (3 - 2 * scaled_t)    
    return f

def smooth_drop (edge0, edge1, t):
    f = np.zeros_like(t)
    f[t<edge0] = 1.0    
    #// Scale/bias into [0..1] range    
    mask = t>=edge0
    mask *= t<=edge1
    scaled_t = (t[mask]-edge0)/(edge1-edge0)    
    f[mask] = 1.0-scaled_t * scaled_t * (3 - 2 * scaled_t)    
    return f


def digital_signal(signal_freq, t, modulation):    
    len_t = t.size
    dsignal0 = smooth_step(0.25, 0.3, t[:int(t.size/2)])
    dsignal1 = smooth_drop(0.7, 0.75, t[int(t.size/2):])
    dsignal = np.concatenate([dsignal0, dsignal1])
    #dsignal = np.zeros_like(t)
    #dsignal[t>0.25] = modulation
    #dsignal[t>0.75] = 0.
    return dsignal

signal = digital_signal(signal_freq, t, modulation)
plt.sca(ax1)
plt.plot(t, signal)

combined_signal = combine_signals(carrier, signal)
noisy_signal = add_noise(combined_signal, 0.0)
plt.sca(ax2)
plt.plot(t, noisy_signal)


#exp_data = noisy_signal
#model_inputs = [ang_freq, t, 0.0]
#res = scipy.optimize.least_squares(error_func, [0.0], args=(model_inputs, exp_data))
#print(res['x'])
#reconstructed_signal =  modulation_signal(ang_freq, t, res['x'])
#plt.plot(t, 1+reconstructed_signal, ls='--')
#plt.plot(t, -(1+reconstructed_signal), ls='--')

plt.sca(ax3)
demodulated_signal = demodulate(noisy_signal, carrier)
plt.plot(t, demodulated_signal)

In [ ]:
t = np.linspace(0, 1, 1000, False)  # 1 second
sig = np.sin(2*np.pi*10*t) + np.sin(2*np.pi*20*t)
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(t, sig)
ax1.set_title('10 Hz and 20 Hz sinusoids')
ax1.axis([0, 1, -2, 2])
sos = scipy.signal.butter(10, 15, 'hp', fs=1000, output='sos')
filtered = scipy.signal.sosfilt(sos, sig)
ax2.plot(t, filtered)
ax2.set_title('After 15 Hz high-pass filter')
ax2.axis([0, 1, -2, 2])
ax2.set_xlabel('Time [seconds]')
plt.tight_layout()


In [ ]:
ang_freq = 1.
t_step = 10/freq
word = "1001"
for iletter, letter in enumerate(word):
    t = np.linspace(t_step*iletter, t_step*(1+iletter), 1000)

    carrier = carrier_signal(ang_freq, t)
    plt.sca(ax0)
    plt.plot(t, carrier)
    if letter == "0":
        modulation = 0.
    if letter == "1":
        modulation = .5
        
    signal = modulation_signal(ang_freq, t, modulation)
    plt.sca(ax1)
    plt.plot(t, signal)

    combined_signal = combine_signals(carrier, signal)
    plt.sca(ax2)
    plt.plot(t, add_noise(combined_signal, 0.5))
    plt.plot(t, 1+signal, ls='--')
    plt.plot(t, -(1+signal), ls='--')
    

In [ ]:
def str_to_binary(test_str):
    converted = list(f"{ord(i):08b}" for i in test_str)
    return converted

def binary_to_str(binary_text_list):
    decoded_letters = []
    for binary_text in binary_text_list:
        binary_as_int = int(binary_text, 2)
        byte_number = ( binary_as_int.bit_length() + 7 ) // 8        
        plaintext_bytes_string = binary_as_int.to_bytes(byte_number, "big")
        unicodeTextString = plaintext_bytes_string.decode() # as UTF-8 standard        
        print(binary_as_int, byte_number, plaintext_bytes_string, unicodeTextString)
        decoded_letters.append(unicodeTextString)    
    return "".join(decoded_letters)

def flip(letter):
    if letter == '0':
        return '1'
    elif letter == '1':
        return '0'

def scramble(binary_text_list):
    scrambled_letters = []
    for binary_text in binary_text_list:
        scrambled = ""
        for iletter, one_zero in enumerate(binary_text):
            if np.random.random() > 0.9 and iletter > 4:
                one_zero = flip(one_zero)
            scrambled += one_zero
        scrambled_letters.append(scrambled)
    return scrambled_letters

#mystr = "hello"
mystr = "".join(("h","e","l","l","o"))
print(mystr)
binary_rep = str_to_binary(mystr)
print(binary_rep)
binary_rep = scramble(binary_rep)
print(binary_rep)
new_str = binary_to_str(binary_rep)
print("[{}]".format(new_str))
#print(" ".join(converted))

#for bnumber in converted:
#    print("{:x}".format(int(bnumber,2)).decode('hex'))

#('%x' % int("".join(converted), 2)).decode('hex').decode('utf-8')
    

In [ ]:
binary_rep[0][0]

In [ ]:
plaintext = "100111010101010101010101010101"
print(type(plaintext)) # gives: <class 'str'>

# make an integer value out of the Unicode string with '0's and '1's: 
plaintextAsInteger = int(plaintext, 2)
byteNumber = ( plaintextAsInteger.bit_length() + 7 ) // 8
print(plaintextAsInteger, byteNumber) # gives: 659903829 4

plaintextBytesString = plaintextAsInteger.to_bytes(byteNumber, "big")
print(type(plaintextBytesString), plaintextBytesString) # gives <class 'bytes'> b"'UUU"

# Then text to unicode
unicodeTextString = plaintextBytesString.decode() # as UTF-8 standard
plaintext = unicodeTextString
print(type(plaintext), len(plaintext), plaintext) # gives: <class 'str'> 4 'UUU

# Then text to a list ( a one dimensional array )
plaintextArray = [ord(c) for c in plaintext]
print(plaintextArray) # gives: [39, 85, 85, 85]

# let's show that the code gives values > 255 (Unicode code)
plaintextArray = [ord(c) for c in "æœ©®"] 
print(plaintextArray) # gives: [230, 339, 169, 174]

In [ ]:
wvl = 1500e-6
freq = constants.c*2*np.pi/(wvl)
print("{:.3f} GHz".format(freq/(2*np.pi*1e9)))
fig, [ax0, ax1, ax2] = plt.subplots(3,1, figsize=(16,12))


def carrier_signal(frequency, time):
    return np.sin(2*np.pi*freq*time) 

def modulation_signal(frequency, time, modulation):
    phase = np.pi/4.
    #modulation = modulation*np.concatenate([np.linspace(0., 1., int((time.size)/2)),
    #                                        np.linspace(1., 0., int((time.size)/2))])
    signal = modulation*np.sin(2*np.pi*(freq/5)*time+phase)
    return signal

def combine_signals(signal1, signal2):
    combined_signal = (1+signal2)*signal1
    return combined_signal

def add_noise(signal, noise_level):
    noise = np.random.normal(0, noise_level, signal.size).reshape(signal.shape)
    return signal+noise


ang_freq = 1.
t_step = 10/freq
word = "1"

def err_func()

for iletter, letter in enumerate(word):
    t = np.linspace(t_step*iletter, t_step*(1+iletter), 1000)

    carrier = carrier_signal(ang_freq, t)
    plt.sca(ax0)
    plt.plot(t, carrier)
    if letter == "0":
        modulation = 0.
    if letter == "1":
        modulation = .5
        
    signal = modulation_signal(ang_freq, t, modulation)
    plt.sca(ax1)
    plt.plot(t, signal)

    combined_signal = combine_signals(carrier, signal)
    plt.sca(ax2)
    plt.plot(t, add_noise(combined_signal, 0.5))
    plt.plot(t, 1+signal, ls='--')
    plt.plot(t, -(1+signal), ls='--')
    